# Transcribe Recordings
paid service [Goodtape](http://goodtape.io) will be used for the transcriptions

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
import os
import docx

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *
from lib_henryk.utils import *

from lib_henryk.recordings import transcription

## Params

In [3]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

# retrieve new token by opening 'https://webhook.site/' in incognito mode
# and picking up the token from the URL
WEBHOOKS_TOKEN_ID = 'f7477993-66da-4747-b9c8-6bc5c302ef3c'
GOODTAPE_API_KEY = os.getenv("GOODTAPE_API_KEY")

## Load Dataset

In [4]:
df = pd.read_csv(FILE_AUDIO_STATS_CSV)
df_transcriptions = df.copy()
df_transcriptions['transcription_id'] = ''

In [20]:
# load sync dataframe
if os.path.exists(FILE_TRANSCRIPTION_CSV):
    print(f'transcription log exists, reading it from {FILE_TRANSCRIPTION_CSV}')
    df_transcriptions = pd.read_csv(FILE_TRANSCRIPTION_CSV)

transcription log exists, reading it from ../../data/processed/henryk_audio_transcription.csv


## Run Transcriptions

In [21]:
# perform transcriptions
transcription.submit_transcriptions_goodtape(
    df=df_transcriptions, 
    webhooks_token_id=WEBHOOKS_TOKEN_ID,
    goodtape_api_key=GOODTAPE_API_KEY,
    path_recordings=DIR_WIADOMOSCI_DO_HENRYCZKA,
    start=0, stop=500,
    verbose=False
)

submitted 0 recordings to the transcription service GoodTape


## Fetch Transcriptions JSONs

In [7]:
# need to wait until transcriptions are available
exit_cell()

StopExecution: stopped

In [23]:
# retrieve goodtape server responses from webhooks
transcription.retrieve_responses_goodtape_via_webhooks(
    df=df_transcriptions,
    webhooks_token_id=WEBHOOKS_TOKEN_ID, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON
)

# process json to text
transcription.process_transcriptions_json(
    df=df_transcriptions, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
    to_doc=True,
    to_txt=True,
    verbose=False,
)

# process json to doc

found 0 requests to fetch
0 transcriptions were retrieved
498 new transcriptions were processed, there are 498 txt and 498 doc transcriptions available


## Cleanup missing requests
- transcription ids for the requests that were lost will be removed, only those that were processed will remain

In [24]:
df_transcriptions

,Unnamed: 0,path,file,name,kind,date,type,duration,transcription_json,transcription_txt,transcription_doc,transcription_id
0,0,2022/Henryk 2022-05-26 Bajka o nowym samochodz...,Henryk 2022-05-26 Bajka o nowym samochodziku.m4a,Bajka o nowym samochodziku,Henryk,26/05/2022,m4a,466.688000,Henryk 2022-05-26 Bajka o nowym samochodziku.json,Henryk 2022-05-26 Bajka o nowym samochodziku.txt,Henryk 2022-05-26 Bajka o nowym samochodziku.docx,None
1,1,2022/Henryk 2022-05-27 Bajka o Eryku r1.m4a,Henryk 2022-05-27 Bajka o Eryku r1.m4a,Bajka o Eryku r1,Henryk,27/05/2022,m4a,546.154667,Henryk 2022-05-27 Bajka o Eryku r1.json,Henryk 2022-05-27 Bajka o Eryku r1.txt,Henryk 2022-05-27 Bajka o Eryku r1.docx,None
2,2,2022/Henryk 2022-05-28 Bajka o swince peppie i...,Henryk 2022-05-28 Bajka o swince peppie i domu...,Bajka o swince peppie i domu rebeki,Henryk,28/05/2022,m4a,387.264000,Henryk 2022-05-28 Bajka o swince peppie i domu...,Henryk 2022-05-28 Bajka o swince peppie i domu...,Henryk 2022-05-28 Bajka o swince peppie i domu...,None
3,3,2022/Henryk 2022-05-30 Bajka o parku dinozauro...,Henryk 2022-05-30 Bajka o parku dinozaurow.m4a,Bajka o parku dinozaurow,Henryk,30/05/2022,m4a,511.658667,Henryk 2022-05-30 Bajka o parku dinozaurow.json,Henryk 2022-05-30 Bajka o parku dinozaurow.txt,Henryk 2022-05-30 Bajka o parku dinozaurow.docx,None
4,4,2022/Henryk 2022-05-31 Bajka o drużynie koszy...,Henryk 2022-05-31 Bajka o drużynie koszykowki...,Bajka o drużynie koszykowki,Henryk,31/05/2022,m4a,492.355918,Henryk 2022-05-31 Bajka o drużynie koszykowki...,Henryk 2022-05-31 Bajka o drużynie koszykowki...,Henryk 2022-05-31 Bajka o drużynie koszykowki...,None
...,...,...,...,...,...,...,...,...,...,...,...,...
493,493,Henryk 2024-04-26 Tatuś opowiada o zebraniu w...,Henryk 2024-04-26 Tatuś opowiada o zebraniu w...,Tatuś opowiada o zebraniu w żłobku a Kamptoz...,Henryk,26/04/2024,m4a,1106.730667,Henryk 2024-04-26 Tatuś opowiada o zebraniu w...,Henryk 2024-04-26 Tatuś opowiada o zebraniu w...,Henryk 2024-04-26 Tatuś opowiada o zebraniu w...,None
494,494,Henryk 2024-04-28 Tatuś opowiada o malowaniu ...,Henryk 2024-04-28 Tatuś opowiada o malowaniu ...,Tatuś opowiada o malowaniu krzesełek a Diplod...,Henryk,28/04/2024,m4a,1025.898667,Henryk 2024-04-28 Tatuś opowiada o malowaniu ...,Henryk 2024-04-28 Tatuś opowiada o malowaniu ...,Henryk 2024-04-28 Tatuś opowiada o malowaniu ...,None
495,495,Henryk 2024-04-29 Tata opowiada o konkursie i ...,Henryk 2024-04-29 Tata opowiada o konkursie i ...,Tata opowiada o konkursie i nagrodach a Gastor...,Henryk,29/04/2024,m4a,1078.997333,Henryk 2024-04-29 Tata opowiada o konkursie i ...,Henryk 2024-04-29 Tata opowiada o konkursie i ...,Henryk 2024-04-29 Tata opowiada o konkursie i ...,None
496,496,Henryk 2024-05-01 Tatuś opowiada o dniu pracy...,Henryk 2024-05-01 Tatuś opowiada o dniu pracy...,Tatuś opowiada o dniu pracy a Trąbiący dino...,Henryk,01/05/2024,m4a,1227.306667,Henryk 2024-05-01 Tatuś opowiada o dniu pracy...,Henryk 2024-05-01 Tatuś opowiada o dniu pracy...,Henryk 2024-05-01 Tatuś opowiada o dniu pracy...,None


In [25]:
# cleanup stale transcription ids - but work on a copy of the log
df_transcriptions_to_write = transcription.get_cleaned_up_transcriptions(
    df=df_transcriptions, 
    path_transcriptions_json=DIR_TRANSCRIPTIONS_JSON, 
    path_transcriptions_txt=DIR_TRANSCRIPTIONS_TXT,
    path_transcriptions_doc=DIR_TRANSCRIPTIONS_DOC,
)

# save transcription log
df_transcriptions_to_write.to_csv(FILE_TRANSCRIPTION_CSV, index=False)

there are 498 processed and valid transcriptions
*** all 498 recordings were transcribed ***
